# **Manipulación de las imágenes**

In [ ]:
import rasterio
import matplotlib.pyplot as plt
from rasterio.plot import show
import numpy as np
import os
from matplotlib.pyplot import figure
from PIL import Image 

In [ ]:
# Directorio de las imagenes
data_dir = "/home/fortanel/Escuela/Tesis/Imagenes/CUADROS-8/"

#Obtiene una lista de los archivos en la ruta proporcionada
imagenes = next(os.walk("/home/fortanel/Escuela/Tesis/Imagenes/CUADROS-8/"))[2]
print(imagenes[15].replace(".tif",""))
#Concatenación dedel directorio y el nombre del archivo con el que se desea trabajar
fp = os.path.join(data_dir, imagenes[15])

#Abrir la imagen
raster = rasterio.open(fp)

#Se obtiene información de la imagene
raster.meta

## **Imagenes rgb**

Las imagenes con las que trabajamos cuentan con 8 bandas. Las bandas 4, 3 y 2 corresponden con los colores rojo, verde y azul respectivamente. Es posible combinar estas bandas para generar una imagen rgb.

In [ ]:
# La función read permite obtener los distintas bandas de la imagen, se obtiene un numpy array.
red = raster.read(4)
green = raster.read(3)
blue = raster.read(2)

#Normaliza los valores de los arreglos
def normalize(array):
    """Normalizes numpy arrays into scale 0.0 - 1.0"""
    array_min, array_max = array.min(), array.max()
    return ((array - array_min)/(array_max - array_min))

redn = normalize(red)
greenn = normalize(green)
bluen = normalize(blue)


# Se apilan los arreglos para forma uno solo de dimension 3, el cual puede ser interpretado por 
#la libreria pyplot como una imagen.
rgb = np.dstack((redn,greenn , bluen))

#La función figure permite establecer el tamaño y calidad de la imagen al ser mostrada, entre otras opciones
figure(num=None, figsize=(20, 20), dpi=80, facecolor='w', edgecolor='k')
#Muestra la imagen
plt.imshow(rgb)

In [ ]:
#Definimos una función para generar estas imágenes
def rgb(dir,landsat):
    #Esto es debido a que las imagenes obtenidas del concurso dstl son de un saltelite disinto
    if (landsat):
        ban = [4,3,2]
    else:
        ban = [5,3,2]
        
    imagen = rasterio.open(dir)
    red = imagen.read(ban[0])
    green = imagen.read(ban[1])
    blue = imagen.read(ban[2])

    redn = normalize(red)
    greenn = normalize(green)
    bluen = normalize(blue)

    rgb = np.dstack((redn,greenn , bluen))
    return rgb
    

## **Índice de Vegetación de Diferencia Normalizada**

Tambien es posible combinar otras bandas, para obtener nuevos índices como el  Índice de Vegetación de Diferencia Normalizada(NDVI). Recordemos que este indice se calcula de la siguiente manera: $$NDVI= \frac{IRCercano - Rojo}{IRCercano + Rojo}$$ Donde IRCercano se refiere a la banda correspondientes al infrarrojo cercano y el Rojo a la banda correspondiente al rojo. En el caso de las imágenes con las que trabajamos la banda 5 corresponde al infrarrojo cercano y la 4 a el rojo. 

In [ ]:
#Obtenemos las bandas 5 y 4
b4 = raster.read(4)
b5 = raster.read(5)

#Obtenemos los valores que generan al NDVI
np.seterr(divide='ignore', invalid='ignore')
ndvi = (b5.astype(float) - b4.astype(float)) / (b5 + b4)

#Mostramos la imagen
figure(num=None, figsize=(20, 20), dpi=80, facecolor='w', edgecolor='k')
plt.imshow(ndvi)

#Genera una barra que indica los valores correspondeintes a los colores
plt.colorbar()

# **Pipeline**

In [ ]:
import os
import random
import pandas as pd
plt.style.use("ggplot")
%matplotlib inline

from tqdm import tqdm_notebook, tnrange
from itertools import chain
from skimage.io import imread, imshow, concatenate_images
from skimage.transform import resize
from skimage.morphology import label
from sklearn.model_selection import train_test_split

import tensorflow as tf

from keras.models import Model, load_model
from keras.layers import Input, BatchNormalization, Activation, Dense, Dropout
from keras.layers.core import Lambda, RepeatVector, Reshape
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D, GlobalMaxPool2D
from keras.layers.merge import concatenate, add
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [ ]:
#Definimos la ruta  donde se encuentran las imagenes
directorio = "/home/fortanel/Escuela/Tesis/dstl/imagenes/"

#Definimos el tamaño de las imágenes con las que se trabajaran.
tam = 128

In [ ]:
#Obtenemos el nombre de las imagenes con las que se trabajaran
ids = next(os.walk(directorio +"3/"))[2]
ids.remove("6010_4_2.tif")
ids

A continuacion mostramos información sobre las etiquetas y las imagenes con las que se trabajara.
Notese que las etiquetas no se encuentran a la misma escala que las imagenes.

In [ ]:
#La etiqueta, se convierte para que solo tenga un color
img_to_array(load_img(directorio + "mascara/" + ids[18].replace(".tif" , ".png"), grayscale=True)).shape

In [ ]:
#Imagen
rasterio.open(directorio + "16/" + ids[18].replace(".tif" , "_M.tif")).meta

In [ ]:
#Comparamos los datos de entrenamiento con sus etiquetas
figure(num=None, figsize=(20, 20), dpi=80, facecolor='w', edgecolor='k')
plt.imshow(rgb( directorio + "16/" + ids[18].replace(".tif" , "_M.tif"),True))

#Abrimos las etiquetas, en esta las etiquetas estan en formato png
Image.open(directorio + "mascara/" + ids[18].replace(".tif",".png")).convert("L")

In [ ]:
#Generamos las variables donde se guardaran las datos de entrenamiento junto a sus etiquetas
X = np.zeros((864,tam,tam,8), dtype=np.float32)
y = np.zeros((864, tam,tam,1), dtype=np.float32)

In [ ]:
#Este paquete tambien permite trabajar con las imágenes en formato tif
import tifffile as tiff
import cv2

#En esta parte del codigo el objetivo es cortar las imágenes con sus respectivas etiquetas en cuadros 
#de 128x128
i = 0
for n, id_ in tqdm_notebook(enumerate(ids), total=1):
    # Cargamos las imagenes
    img = tiff.imread('dstl/imagenes/16/'+ id_.replace(".tif","_M.tif") )
    
    #Cargamos las etiquetas y las convertimos a array
    mask = img_to_array(load_img("dstl/imagenes/mascara/" + id_.replace(".tif",".png"), grayscale=True))
    #Esta parte del codigo nos permite reducir el tamaño de las etiquetas para que coincidan con 
    #las imagenes, manteniendo sus proporciones.
    mask = cv2.resize(mask, dsize = (int(len(mask)/4),int(len(mask[0])/4)))
    #Omitimos los datos innecesario
    mask = mask.reshape(mask.shape[0],mask.shape[1],1)
    #mask = zoom(mask, 0.25)
    #mask = mask.reshape((len(mask),len(mask[0]) , 1))
    j = 0
    s = 0
    
    #Cortamos las imágenes en cuadros de 128*128
    while(j < 6):
        k = 0
        while(k < 6):
            aux = img[:,tam*j:tam*(j+1),tam*k:tam*(k+1)]
            aux = aux.reshape(tam,tam,8)
            # Load masks
            aux1 = mask[tam*j:tam*(j+1),tam*k:(tam)*(k+1)]
            # Save images
            X[36*i + 6*j + k ] = aux/255.0
            y[36*i + 6*j + k ] = aux1//255.0
            k = k+1 
        j = j + 1
        
    
    i = i+1

In [ ]:
#Generamos los datos de entrenamiento y validacion
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.1, random_state=42)

In [ ]:
def conv2d_block(input_tensor, n_filters, kernel_size = 3, batchnorm = True):
    """Function to add 2 convolutional layers with the parameters passed to it"""
    # first layer
    x = Conv2D(filters = n_filters, kernel_size = (kernel_size, kernel_size),\
              kernel_initializer = 'he_normal', padding = 'same')(input_tensor)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    # second layer
    x = Conv2D(filters = n_filters, kernel_size = (kernel_size, kernel_size),\
              kernel_initializer = 'he_normal', padding = 'same')(input_tensor)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    return x
#Se define la red en este caso es U-net.  
def get_unet(input_img, n_filters = 16, dropout = 0.1, batchnorm = True):
    # Contracting Path
    c1 = conv2d_block(input_img, n_filters * 1, kernel_size = 3, batchnorm = batchnorm)
    p1 = MaxPooling2D((2, 2))(c1)
    p1 = Dropout(dropout)(p1)
    
    c2 = conv2d_block(p1, n_filters * 2, kernel_size = 3, batchnorm = batchnorm)
    p2 = MaxPooling2D((2, 2))(c2)
    p2 = Dropout(dropout)(p2)
    
    c3 = conv2d_block(p2, n_filters * 4, kernel_size = 3, batchnorm = batchnorm)
    p3 = MaxPooling2D((2, 2))(c3)
    p3 = Dropout(dropout)(p3)
    
    c4 = conv2d_block(p3, n_filters * 8, kernel_size = 3, batchnorm = batchnorm)
    p4 = MaxPooling2D((2, 2))(c4)
    p4 = Dropout(dropout)(p4)
    
    c5 = conv2d_block(p4, n_filters = n_filters * 16, kernel_size = 3, batchnorm = batchnorm)
    
    # Expansive Path
    u6 = Conv2DTranspose(n_filters * 8, (3, 3), strides = (2, 2), padding = 'same')(c5)
    u6 = concatenate([u6, c4])
    u6 = Dropout(dropout)(u6)
    c6 = conv2d_block(u6, n_filters * 8, kernel_size = 3, batchnorm = batchnorm)
    
    u7 = Conv2DTranspose(n_filters * 4, (3, 3), strides = (2, 2), padding = 'same')(c6)
    u7 = concatenate([u7, c3])
    u7 = Dropout(dropout)(u7)
    c7 = conv2d_block(u7, n_filters * 4, kernel_size = 3, batchnorm = batchnorm)
    
    u8 = Conv2DTranspose(n_filters * 2, (3, 3), strides = (2, 2), padding = 'same')(c7)
    u8 = concatenate([u8, c2])
    u8 = Dropout(dropout)(u8)
    c8 = conv2d_block(u8, n_filters * 2, kernel_size = 3, batchnorm = batchnorm)
    
    u9 = Conv2DTranspose(n_filters * 1, (3, 3), strides = (2, 2), padding = 'same')(c8)
    u9 = concatenate([u9, c1])
    u9 = Dropout(dropout)(u9)
    c9 = conv2d_block(u9, n_filters * 1, kernel_size = 3, batchnorm = batchnorm)
    
    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c9)
    model = Model(inputs=[input_img], outputs=[outputs])
    return model

In [ ]:
#Definimos la capata de entrada
input_img = Input((tam, tam,8), name='img')

#Generamos la red con la que se entrenara
model = get_unet(input_img, n_filters=16, dropout=0.05, batchnorm=True)

#Se configura la red
model.compile(optimizer=Adam(), loss="binary_crossentropy", metrics=["accuracy"])
#Esta aprte nos permite configurar el entrenaiento
callbacks = [
    EarlyStopping(patience=10, verbose=1),
    ReduceLROnPlateau(factor=0.1, patience=5, min_lr=0.00001, verbose=1),
    ModelCheckpoint('model-tgs-salt.h5', verbose=1, save_best_only=True, save_weights_only=True)
]

In [ ]:
#Un resumen de la red
model.summary()

In [ ]:
#Entreamos a la red
results = model.fit(X_train, y_train, batch_size=32, epochs=3, callbacks=callbacks,\
                    validation_data=(X_valid, y_valid))

In [ ]:
#Observamos el desempeño de la red
plt.figure(figsize=(8, 8))
plt.title("Learning curve")
plt.plot(results.history["loss"], label="loss")
plt.plot(results.history["val_loss"], label="val_loss")
plt.plot( np.argmin(results.history["val_loss"]), np.min(results.history["val_loss"]), marker="x", color="r", label="best model")
plt.xlabel("Epochs")
plt.ylabel("log_loss")
plt.legend();

In [ ]:
#Obtenemos los pesos de la mejor red
model.load_weights('model-tgs-ds.h5')

In [ ]:
#Evaluamos el desempeño
model.evaluate(X, y,verbose=1)

In [ ]:
#Generamos una predicción
preds_train = model.predict(X, verbose=1)
preds_val = model.predict(X, verbose=1)

In [ ]:
preds_train

In [ ]:
#Esta parte permite convertir la predicción de los pixeles en 0 y 1.
preds_train_t = (preds_train > 0.5).astype(np.uint8)
preds_val_t = (preds_val > 0.5).astype(np.uint8)

In [ ]:
preds_train_t

In [ ]:
#Visualizamos el las predicciones
resultado = np.zeros((tam*6,tam*6,1), dtype=np.float32)
prediccion = np.zeros((tam*6,tam*6,1), dtype=np.float32)
v = 0
while(v < 6):
    w = 0
    while(w < 6):
        resultado[128*v:128*(v+1), 128*w: 128*(w+1),:] =y[(v*6)+w + 36*13]
        prediccion[128*v:128*(v+1), 128*w: 128*(w+1),:] =preds_train_t[(v*6)+w + 36*3]
        w = w+1
    v = v+1  
    
tiff.imshow(resultado, cmap = "gray")
tiff.imshow(prediccion, cmap = "gray")